In [1]:
#%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

import matplotlib
import matplotlib.pyplot as plt
import numpy as np

/Users/oshinibandara/opt/anaconda3/envs/python_3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
transform1 = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.SVHN(root='./data', split = 'train',
                                        download=True, transform=transform1)
trainloader= torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)
train_size1 = int(0.5 * len(trainset))
train_size2 = len(trainset)-int(0.5 * len(trainset))
trainset1, trainset2 = torch.utils.data.random_split(trainset, [train_size1, train_size2])

trainloader1= torch.utils.data.DataLoader(trainset1, batch_size=4,
                                          shuffle=True, num_workers=2)
trainloader2 = torch.utils.data.DataLoader(trainset2, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.SVHN(root='./data', split='test',
                                       download=True, transform=transform1)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)
print(len(trainset))
print(len(trainset1))
print(len(trainset2))
print(len(testset))


classes = ('1','2','3','4','5','6','7','8','9','0')


Using downloaded and verified file: ./data/train_32x32.mat
Using downloaded and verified file: ./data/test_32x32.mat
73257
36628
36629
26032


In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, len(classes))

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [5]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader1, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training train set 1')

[1,  2000] loss: 2.247
[1,  4000] loss: 2.225
[1,  6000] loss: 2.050
[1,  8000] loss: 1.242
[2,  2000] loss: 0.683
[2,  4000] loss: 0.618
[2,  6000] loss: 0.568
[2,  8000] loss: 0.565
Finished Training train set 1


In [6]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy1= float(correct)/float(total)*100
print('Accuracy of the network on the 26032 test images: %.2f ' % accuracy1)

Accuracy of the network on the 26032 test images: 82.48 


In [7]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
correct_indicator = [0 for i in range(len(testset))]
i = 0
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            
            if label == prediction:
                correct_pred[classes[label]] += 1
                correct_indicator[i] = 1
            total_pred[classes[label]] += 1
            i += 1

# 1 - 1, 2 - 1, 3 - 1, 4-0, 5-0, 6-0 - 60
# 2, 3, 4, 5 - 80
print(correct_indicator[0:20])

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

[1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy for class: 1     is 73.1 %
Accuracy for class: 2     is 92.6 %
Accuracy for class: 3     is 85.9 %
Accuracy for class: 4     is 78.0 %
Accuracy for class: 5     is 81.0 %
Accuracy for class: 6     is 85.9 %
Accuracy for class: 7     is 68.1 %
Accuracy for class: 8     is 85.9 %
Accuracy for class: 9     is 74.0 %
Accuracy for class: 0     is 79.2 %


In [8]:
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader2, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training train set 2')

[1,  2000] loss: 0.535
[1,  4000] loss: 0.496
[1,  6000] loss: 0.486
[1,  8000] loss: 0.461
[2,  2000] loss: 0.412
[2,  4000] loss: 0.408
[2,  6000] loss: 0.414
[2,  8000] loss: 0.388
Finished Training train set 2


In [9]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
accuracy2= float(correct)/float(total)*100


print('Accuracy of the network on the 26032 test images: %.2f ' % accuracy2)

Accuracy of the network on the 26032 test images: 85.28 


In [10]:
# prepare to count predictions for each class
correct_pred = {classname: 0 for classname in classes}
total_pred = {classname: 0 for classname in classes}
correct_indicator = [0 for i in range(len(testset))]
i = 0
# again no gradients needed
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predictions = torch.max(outputs, 1)
        # collect the correct predictions for each class
        for label, prediction in zip(labels, predictions):
            
            if label == prediction:
                correct_pred[classes[label]] += 1
                correct_indicator[i] = 1
            total_pred[classes[label]] += 1
            i += 1

# 1 - 1, 2 - 1, 3 - 1, 4-0, 5-0, 6-0 - 60
# 2, 3, 4, 5 - 80
print(correct_indicator[0:20])

# print accuracy for each class
for classname, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[classname]
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Accuracy for class: 1     is 81.9 %
Accuracy for class: 2     is 92.7 %
Accuracy for class: 3     is 89.4 %
Accuracy for class: 4     is 68.6 %
Accuracy for class: 5     is 88.0 %
Accuracy for class: 6     is 85.3 %
Accuracy for class: 7     is 85.1 %
Accuracy for class: 8     is 87.7 %
Accuracy for class: 9     is 76.4 %
Accuracy for class: 0     is 86.8 %


In [11]:
forgetfullness = (accuracy1-accuracy2)
print('forgetting percentage of the network is: %f %%' % forgetfullness)

forgetting percentage of the network is: -2.804241 %
